#  Train a GPT-2 Text-Generating Model w/ GPU For Free 

FORK of the original notebook:

https://colab.research.google.com/drive/1VLG8e7YSEwypxU-noRNhsv5dW4NfTGce

by [Max Woolf](http://minimaxir.com)

*Last updated: November 10th, 2019*

Retrain an advanced text generating neural network on any text dataset **for free on a GPU using Collaboratory** using `gpt-2-simple`!

For more about `gpt-2-simple`, you can visit [this GitHub repository](https://github.com/minimaxir/gpt-2-simple). You can also read my [blog post](https://minimaxir.com/2019/09/howto-gpt2/) for more information how to use this notebook!


In [2]:
!pip install -q gpt-2-simple
import gpt_2_simple as gpt2
from datetime import datetime

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



## GPU

Colaboratory uses either a Nvidia T4 GPU or an Nvidia K80 GPU. The T4 is slightly faster than the old K80 for training GPT-2, and has more memory allowing you to train the larger GPT-2 models and generate more text.

You can verify which GPU is active by running the cell below.

In [2]:
!nvidia-smi

Tue Nov 17 01:31:31 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.80.02    Driver Version: 450.80.02    CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce GTX 1080    Off  | 00000000:01:00.0 Off |                  N/A |
|  0%   33C    P8     6W / 180W |     11MiB /  8119MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Downloading GPT-2

If you're retraining a model on new text, you need to download the GPT-2 model first. 

There are three released sizes of GPT-2:

* `124M` (default): the "small" model, 500MB on disk.
* `355M`: the "medium" model, 1.5GB on disk.
* `774M`: the "large" model, cannot currently be finetuned with Colaboratory but can be used to generate text from the pretrained model (see later in Notebook)
* `1558M`: the "extra large", true model. Will not work if a K80 GPU is attached to the notebook. (like `774M`, it cannot be finetuned).

Larger models have more knowledge, but take longer to finetune and longer to generate text. You can specify which base model to use by changing `model_name` in the cells below.

The next cell downloads it from Google Cloud Storage and saves it in the Colaboratory VM at `/models/<model_name>`.

This model isn't permanently saved in the Colaboratory VM; you'll have to redownload it if you want to retrain it at a later time.

In [3]:
gpt2.download_gpt2(model_name="124M")

Fetching checkpoint: 1.05Mit [00:00, 544Mit/s]                                                      
Fetching encoder.json: 1.05Mit [00:00, 2.69Mit/s]                                                   
Fetching hparams.json: 1.05Mit [00:00, 1.42Git/s]                                                   
Fetching model.ckpt.data-00000-of-00001: 498Mit [01:19, 6.24Mit/s]                                  
Fetching model.ckpt.index: 1.05Mit [00:00, 274Mit/s]                                                
Fetching model.ckpt.meta: 1.05Mit [00:00, 5.70Mit/s]                                                
Fetching vocab.bpe: 1.05Mit [00:00, 5.67Mit/s]                                                      


## Get file from your Google Drive

The best way to get input text to-be-trained into the Colaboratory VM, and to get the trained model *out* of Colaboratory, is to route it through Google Drive *first*.

In [3]:
!gdown --id 0BzNAHGvG5wxZZjcwMDc5YjctNGI4OC00YzAxLWJhYTctNGQ3ZDNmMmZlMjNh

Downloading...
From: https://drive.google.com/uc?id=0BzNAHGvG5wxZZjcwMDc5YjctNGI4OC00YzAxLWJhYTctNGQ3ZDNmMmZlMjNh
To: /home/guma/Documents/App/ML/datacrunch-stylegan2-ada/shakespeare.txt
5.58MB [00:00, 6.40MB/s]


In [4]:
file_name = "shakespeare.txt"

## Finetune GPT-2

The next cell will start the actual finetuning of GPT-2. It creates a persistent TensorFlow session which stores the training config, then runs the training for the specified number of `steps`. (to have the finetuning run indefinitely, set `steps = -1`)

The model checkpoints will be saved in `/checkpoint/run1` by default. The checkpoints are saved every 500 steps (can be changed) and when the cell is stopped.

The training might time out after 4ish hours; make sure you end training and save the results so you don't lose them!

**IMPORTANT NOTE:** If you want to rerun this cell, **restart the VM first** (Runtime -> Restart Runtime). You will need to rerun imports but not recopy files.

Other optional-but-helpful parameters for `gpt2.finetune`:


*  **`restore_from`**: Set to `fresh` to start training from the base GPT-2, or set to `latest` to restart training from an existing checkpoint.
* **`sample_every`**: Number of steps to print example output
* **`print_every`**: Number of steps to print training progress.
* **`learning_rate`**:  Learning rate for the training. (default `1e-4`, can lower to `1e-5` if you have <1MB input data)
*  **`run_name`**: subfolder within `checkpoint` to save the model. This is useful if you want to work with multiple models (will also need to specify  `run_name` when loading the model)
* **`overwrite`**: Set to `True` if you want to continue finetuning an existing model (w/ `restore_from='latest'`) without creating duplicate copies. 

In [5]:
sess = gpt2.start_tf_sess()

gpt2.finetune(sess,
              dataset=file_name,
              model_name='124M',
              steps=1000,
              restore_from='fresh',
              run_name='run1',
              print_every=10,
              sample_every=200,
              save_every=500
              )

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Loading checkpoint models/124M/model.ckpt
INFO:tensorflow:Restoring parameters from models/124M/model.ckpt


  0%|          | 0/1 [00:00<?, ?it/s]

Loading dataset...


100%|██████████| 1/1 [00:04<00:00,  4.76s/it]


dataset has 1850683 tokens
Training...
[10 | 22.25] loss=3.04 avg=3.04
[20 | 40.58] loss=2.80 avg=2.92
[30 | 58.92] loss=2.67 avg=2.84
[40 | 77.26] loss=2.86 avg=2.84
[50 | 95.63] loss=2.92 avg=2.86
[60 | 113.99] loss=2.86 avg=2.86
[70 | 132.35] loss=2.81 avg=2.85
[80 | 150.72] loss=2.84 avg=2.85
[90 | 169.07] loss=2.72 avg=2.83
[100 | 187.43] loss=2.66 avg=2.82
[110 | 205.80] loss=2.60 avg=2.80
[120 | 224.18] loss=2.84 avg=2.80
[130 | 242.54] loss=2.74 avg=2.79
[140 | 260.91] loss=2.96 avg=2.81
[150 | 279.27] loss=2.63 avg=2.79
[160 | 297.64] loss=2.82 avg=2.80
[170 | 316.02] loss=2.75 avg=2.79
interrupted
Saving checkpoint/run1/model-177


After the model is trained, you can save the checkpoint folder to your local computer then to Google Drive.

You're done! Feel free to go to the **Generate Text From The Trained Model** section to generate text based on your retrained model.

## Load a Trained Model Checkpoint

Running the next cell will copy the `.rar` checkpoint file from your Google Drive into the Colaboratory VM.

In [ ]:
gpt2.copy_checkpoint_from_gdrive(run_name='run1')

The next cell will allow you to load the retrained model checkpoint + metadata necessary to generate text.

**IMPORTANT NOTE:** If you want to rerun this cell, **restart the VM first** (Runtime -> Restart Runtime). You will need to rerun imports but not recopy files.

In [ ]:
sess = gpt2.start_tf_sess()
gpt2.load_gpt2(sess, run_name='run1')

## Generate Text From The Trained Model

After you've trained the model or loaded a retrained model from checkpoint, you can now generate text. `generate` generates a single text from the loaded model.

In [ ]:
gpt2.generate(sess, run_name='run1')

A patient and a sworn enemy,
To one manly gentleman helps to make off
One fairer feather: so, happily, I thank thee.

HENRY BOLINGBROKE:
Welcome, Harry; welcome, Somerset: but that
Me too shall be revenged on you.

WARWICK:
O happy friend I had, as it were a curse,
To leave his country of so fair a life!

SOMERSET:
What foul beast is that in your chamber,
Whose big ears and proud eyes shall slander thee
For so blasphemous a charge?

MONTAGUE:
If I thought this was the sweetest, freshest, and lightest
In all my cup of wine, I should be silent.
It drinks me full of the sweets I drink,
And makes me smile more than the sweetest flower.
The more I drink, the more full I feel; and so
The more sweet is the sweet drink I feel,
And the more full full I feel.

KING HENRY VI:
Welcome to London, Somerset, and Lancaster.
Welcome, cousin, to this comfortable bed;
So naked and loathsome are these theots!

QUEEN MARGARET:
What, shall I waken a new man in my sight?

KING HENRY VI:
What, shall I waken a

If you're creating an API based on your model and need to pass the generated text elsewhere, you can do `text = gpt2.generate(sess, return_as_list=True)[0]`

You can also pass in a `prefix` to the generate function to force the text to start with a given character sequence and generate text from there (good if you add an indicator when the text starts).

You can also generate multiple texts at a time by specifing `nsamples`. Unique to GPT-2, you can pass a `batch_size` to generate multiple samples in parallel, giving a massive speedup (in Colaboratory, set a maximum of 20 for `batch_size`).

Other optional-but-helpful parameters for `gpt2.generate` and friends:

*  **`length`**: Number of tokens to generate (default 1023, the maximum)
* **`temperature`**: The higher the temperature, the crazier the text (default 0.7, recommended to keep between 0.7 and 1.0)
* **`top_k`**: Limits the generated guesses to the top *k* guesses (default 0 which disables the behavior; if the generated output is super crazy, you may want to set `top_k=40`)
* **`top_p`**: Nucleus sampling: limits the generated guesses to a cumulative probability. (gets good results on a dataset with `top_p=0.9`)
* **`truncate`**: Truncates the input text until a given sequence, excluding that sequence (e.g. if `truncate='<|endoftext|>'`, the returned text will include everything before the first `<|endoftext|>`). It may be useful to combine this with a smaller `length` if the input texts are short.
*  **`include_prefix`**: If using `truncate` and `include_prefix=False`, the specified `prefix` will not be included in the returned text.

In [ ]:
gpt2.generate(sess,
              length=250,
              temperature=0.7,
              prefix="LORD",
              nsamples=5,
              batch_size=5
              )

LORD WILLOUGHBY:
That, by the way, Clarence and I have done good side by side;
And yet side we, and he side we have done ill.

KING RICHARD II:
Why then 'tis done ill. O, how should I ease it?
Side with him and my brother, my sovereign!
Side wither away, and as night falls,
Like to the farthest morning to my last,
Side wither away, and as morning comes,
Like to the furthest afternoon to my last!
Side wither away, and as our fortunes turn,
Like to the furthest afternoon to our last!

QUEEN MARGARET:
What is this? counsel? counsel!

KING RICHARD II:
My queen and my heir, for half a mile and a half
She will glide this way, to be or no.

QUEEN MARGARET:
So stands the orchard here, for half a mile and a!

KING RICHARD II:
So stands the orchard here, to fence it, to!
Fashion it in her, like the hedgehog's net
LORD STANLEY:
What if I told you, in the hope of succor,
That I had lain a little while in your arms?

DUKE OF YORK:
No doubt, my lord.

QUEEN ELIZABETH:
'Tis a pity I should be coil'd 

For bulk generation, you can generate a large amount of text to a file and sort out the samples locally on your computer. The next cell will generate a generated text file with a unique timestamp.

You can rerun the cells as many times as you want for even more generated texts!

In [ ]:
gen_file = 'gpt2_gentext_{:%Y%m%d_%H%M%S}.txt'.format(datetime.utcnow())

gpt2.generate_to_file(sess,
                      destination_path=gen_file,
                      length=500,
                      temperature=0.7,
                      nsamples=100,
                      batch_size=20
                      )

In [ ]:
# may have to run twice to get file to download
files.download(gen_file)

## Generate Text From The Pretrained Model

If you want to generate text from the pretrained model, not a finetuned model, pass `model_name` to `gpt2.load_gpt2()` and `gpt2.generate()`.

This is currently the only way to generate text from the 774M or 1558M models with this notebook.

In [7]:
model_name = "774M"

gpt2.download_gpt2(model_name=model_name)

Fetching checkpoint: 1.05Mit [00:00, 296Mit/s]                                                      
Fetching encoder.json: 1.05Mit [00:00, 3.02Mit/s]                                                   
Fetching hparams.json: 1.05Mit [00:00, 378Mit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 3.10Git [05:50, 8.82Mit/s]                                 
Fetching model.ckpt.index: 1.05Mit [00:00, 58.2Mit/s]                                               
Fetching model.ckpt.meta: 2.10Mit [00:00, 7.70Mit/s]                                                
Fetching vocab.bpe: 1.05Mit [00:00, 6.49Mit/s]                                                      


In [3]:
model_name = "774M"
sess = gpt2.start_tf_sess()

gpt2.load_gpt2(sess, model_name=model_name)

Loading pretrained model models/774M/model.ckpt
INFO:tensorflow:Restoring parameters from models/774M/model.ckpt


In [10]:
gpt2.generate(sess,
              model_name=model_name,
              prefix="I have a dream ",
              length=100,
              temperature=0.7,
              top_p=0.9,
              nsamples=5,
              batch_size=5
              )

I have a dream !!! I have a dream that my dream can come true and I can become a writer. I have a dream that I can write. I have a dream that I can become a writer. I have a dream that I can be a writer. I have a dream that I can become a writer. I have a dream that I can become a writer. I have a dream that I can become a writer. I have a dream that I can become a writer. I have a dream that I can
I have a dream !"

A: "Yes, I do."

Q: "What do you dream about?"

A: "My dream is to be able to see the world and know it."

Q: "How do you know what you know?"

A: "Because I know I know."

Q: "How do you know what you know?"

A: "Because I know I know."

Q: "How do you know you know
I have a dream !"

"Dreams, huh?"

"Yes, of course."

The young man's face changed.

It was a face that was full of admiration and love, but at the same time, it was filled with a bitter feeling.

"The people I love, they are all people who are not as strong as me. I am an orphan, and I am not the same as them

# LICENSE

MIT License

Copyright (c) 2019 Max Woolf

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.